Our Goal :
    
    custom_model = myNN()
    custom_model.fit(x_train_scaled, y_train, epochs = 8000, loss_threshold = 0.4631)
    custom_model.predict (X_test_scaled)
    
To do list:

    1. Train_test splitting is to be done manually. Do it later ! - Done
    2. Generic implementation is not done. External parameters are bypassed manually inside the fit() function. 
    Try to solve it! 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("insurance_data.csv")

In [3]:
df

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


 __Splitting data using sklearn just for cross checking. Actual manual splitting is done in the next cells__

__Manually Splitting data__

    Points to remember while splitting data:
    1. Train set can be a pandas dataset
    2. Test set requires the data in the form of pandas series. 
    
    custom_model.fit(x_train_scaled, y_train, epochs = 8000, loss_threshold = 0.4631) requires y_train in pandas series.Make  sure that function returns the values in the correct format.

In [4]:
def split_train_test(data, test_ratio):                      # setting i/p value for func
    np.random.seed(25)                                       # deterministic random generation using 42 - 101010 (expld above)
    shuffled = np.random.permutation(len(data))              # creating shuffled index for total rows of data
    test_set_size= int (len(data)*test_ratio)                # setting size of test data 
    test_indices= shuffled[:test_set_size]                   # spliting test indices from shuffled indices
    train_indices = shuffled[test_set_size:]                 # splitting train indices from shuffled indices
    
    shape = data.shape                                       # Returns tuple (100,3)--> (Rows, columns)
    column_size = shape[1]                                   # Taking 1st index value from returned tuple
    y_index= column_size-1                                   # Generating the column index 

#     return data.iloc[Rowstart:Rowend,columnstart:columnend] # Signature code
#     return data.iloc[train_indices,0:2], data.iloc[test_indices,0:2], data.iloc[train_indices,2:3], data.iloc[test_indices,2:3]
    
    x_train = data.iloc[train_indices,0:y_index]      #Creating pandas dataframe from shuffled indices
    X_test  = data.iloc[test_indices,0:y_index]       #Creating pandas dataframe from shuffled indices
    
    '''Why Pandas series is required for y_train,y_test?
    Fit function accepts y_train as a pandas series only. But why ? 
    Inside fit() --> gradient descent() --> w1d= (1/n)*sum(x1*(y_predicted- y_true))
    --> y_true cannot accept pandas dataframe for calculations.Matrix subtraction requires a 1d array.(y_predicted- y_true)
    Here y_true is derived from y_train which needs to be in a form of series/numbers and not a dataframe.
    
    Here Initially we need to create a pandas dataframe using iloc from the shuffled indexes in the rows.
    iloc(accepts index) loc(accepts actual string/number)
    1. iloc[rows_start : rows_end , column_start : column_end]--> returns dataframe
    2. iloc[rows_start : rows_end , [column_indices]]--> returns dataframe
    3. iloc[[rows_indices], column_start : column_end]--> returns dataframe
    
    4. iloc[rows_start : rows_end,single_column_index] --> Returns pandas series.
    5. iloc[single_column_index,column_start : column_end] --> Returns pandas series.
    
    Update 1: Commented this as I found better option for this solution.
    As we need shuffled index from rows, and index from columns, is the 4th case explained above which is not possible. 
    So initially we converted to pandas dataframe with shuffled indices for rows, and all columns
    Later converting it to pandas series by selecting the desired column from the newly created dataframe.
    Update 2: Better method of converting dataframe to series using squeeze()
    Update 3: 4 and 5 methods shown above returns series directly. No squeeze needed. 
    '''
    
#     y_set = data.iloc[[indices],[column_index]].squeeze()  #General layout Creating pandas series from shuffled indices

#     y_train = data.iloc[train_indices.tolist(),[y_index]]           # Returns pandas dataframe(we require series)
#     y_train = data.iloc[train_indices.tolist(),[y_index]].squeeze() # squeeze() converts dataframe to series 
    y_train = data.iloc[train_indices.tolist(),y_index]             # Using 4/5th option of iloc .
#     y_train = data.iloc[train_indices,y_index:column_size]          #Creating pandas dataframe from shuffled indices
#     y_train = y_train.iloc[:,0]                                     #Creating pandas series as an input for fit()

#     y_test = data.iloc[test_indices.tolist(),[y_index]]               
#     y_test = data.iloc[test_indices.tolist(),[y_index]].squeeze()
    y_test = data.iloc[test_indices.tolist(),y_index]
#     y_test  = data.iloc[test_indices,y_index:column_size]
#     y_test = y_test.iloc[:,0]

    return x_train, X_test, y_train, y_test

#     Note : Keep X in pandas dataframe and y in pandas series.Use the below lines for testing. 
#     return x_train
#     return X_test
#     return y_train
#     return y_test

In [5]:
x_train, X_test, y_train, y_test  = split_train_test(df, 0.22) # Test Ratio adjusted to 0.22 to match the sklearns output after splitting.

In [6]:
type(x_train)  

pandas.core.frame.DataFrame

In [7]:
type(X_test) 

pandas.core.frame.DataFrame

In [8]:
type(y_train)

pandas.core.series.Series

In [9]:
type(y_test)

pandas.core.series.Series

__Scaling Features__
Age and affordability on same scale

In [10]:
x_train_scaled = x_train.copy()
x_train_scaled['age']= x_train_scaled['age']/100
X_test_scaled = X_test.copy()
X_test_scaled['age']= X_test_scaled['age']/100

In [11]:
x_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [12]:
class myNN:
    def __init__(self):
        self.w1= 1      #Defining the initial weights
        self.w2= 1      #Defining the initial weights
        self.bias = 0   #Defining the initial bias
        
    def sigmoid_python(self,X): # Made with precision to take numpy array as an input.
        return (1/ (1 + np.exp(-X)))
    
    def log_loss(self,y_true, y_predicted): # Made with precision to take numpy array as an input.Refer insurance predictor NB for more details.
        epsilon = 1e-15
        y_predicted_new = [max(i,epsilon) for i in y_predicted ]
        y_predicted_new = [min (i, 1-epsilon) for i in y_predicted_new]
        y_predicted_new = np.array(y_predicted_new) 
        return -np.mean(y_true* np.log(y_predicted_new)+ (1-y_true)*np.log(1-y_predicted_new))
    
    '''Passing the x_train_scaled inside the fit function as x and then fine tuning it for the x1 and x2 values of 
    gradient descent function. In tensorflow/keras fit function we cannot bypass such external parameters. Here we are also 
    passing the gradient_descent function inside our fit function to get x1,x2,bias as output, which will further be used
    in the predict function.We could have implemented the complete code of gradient_descent function in fit function directly.
    But since we are bypassing x1 and x2 as x['age'],x['affordibility'], hence writting a separate function for that.'''
    
    def fit(self, x,y,epochs,loss_threshold):   # Returns tuple (x1,x2,bias) to pass in predict function.
        self.w1,self.w2,self.bias= self.gradient_descent(x['age'],x['affordibility'],y,epochs,loss_threshold)
        return self.w1,self.w2,self.bias   # To print w1,w2,w3 passed from gradient_descent function.
    
    def predict(self,x):
        weighted_sum = self.w1*x['age'] + self.w2*x['affordibility']  + self.bias
        return self.sigmoid_python(weighted_sum)
        
    def gradient_descent(self,x1,x2,y_true,epochs,loss_threshold): #Gradient descent function for logistic regression.
#         w1=w2=1 #Taking 1 . Can also take 0 # Initially took these values. Later took it from self.w
#         bias = 0
        rate = 0.5 #usually 0.01
        n = len(x1)
    
        for i in range(epochs):
            weighted_sum = self.w1*x1 + self.w2*x2 +self.bias     # finding weighted sum
            y_predicted = self.sigmoid_python(weighted_sum)  # substituting in sigmoid
            loss = self.log_loss(y_true, y_predicted)        # calculating the loss

            w1d= (1/n)*sum(x1*(y_predicted- y_true))   ## Simplest way to find partial derivative
            w2d= (1/n)*sum(x2*(y_predicted- y_true))   ## Simplest way to find partial derivative 
            bd= np.mean(y_predicted- y_true) 

    #         w1d = (1/n)*np.dot(np.transpose(x1),(y_predicted- y_true))  ## Just another way to find the partial derivative using numpy
    #         w2d = (1/n)*np.dot(np.transpose(x2),(y_predicted- y_true))  ## Just another way to find the partial derivative using numpy

    #         w1d = (1/n)*(y_predicted- y_true).dot(x1.transpose())  ## And one moreway to find the partial derivative using numpy.
    #         w2d = (1/n)*(y_predicted- y_true).dot(x2.transpose())  ## And one moreway to find the partial derivative using numpy.

            self.w1 = self.w1 - rate*w1d  # Updating weights using partial derivative to minimise the loss
            self.w2 = self.w2 - rate*w2d  # Updating weights using partial derivative to minimise the loss
            self.bias = self.bias - rate*bd # Updating bias using partial derivative to minimise the loss
        
            if i%50 == 0: # To print the results after every 50 iterations/epochs.Can also remove this statement completely.
                print (f"epoch : {i}, w1: {self.w1}, w2: {self.w2}, bias: {self.bias}, loss: {loss}")
                
            if loss<=loss_threshold: # Loss can be specified as a threshold. It will break the specified iterations if specified loss is reached.
                print (f"epoch : {i}, w1: {self.w1}, w2: {self.w2}, bias: {self.bias}, loss: {loss}") #Threshold satisfied at 364 iteration. As we have set the iterations to print at intervals of 50, It breaks at 350.So 364th iteration is not printed. To avoid that we print the epoch line right after we get the threshold loss at line 364.
                break
                
    #         print(y_predicted) 

        return self.w1,self.w2,self.bias # To pass in fit function.


__Note : Fit function accepts test data as a pandas series only. Make sure to convert pandas Dataframe to pandas series before supplying.__ Reason explained above in train_test_split()

In [13]:
custom_model = myNN()
custom_model.fit(x_train_scaled, y_train, epochs = 8000, loss_threshold = 0.4631)

epoch : 0, w1: 0.974907633470177, w2: 0.948348125394529, bias: -0.11341867736368583, loss: 0.7113403233723417
epoch : 50, w1: 1.503319554173139, w2: 1.108384790367645, bias: -1.2319047301235464, loss: 0.5675865113475955
epoch : 100, w1: 2.200713131760032, w2: 1.2941584023238903, bias: -1.6607009122062801, loss: 0.5390680417774752
epoch : 150, w1: 2.8495727769689085, w2: 1.3696895491572747, bias: -1.986105845859897, loss: 0.5176462164249293
epoch : 200, w1: 3.4430169708818044, w2: 1.4042218624465033, bias: -2.2571369883752728, loss: 0.5005011269691374
epoch : 250, w1: 3.9824504946495773, w2: 1.423912732932123, bias: -2.494377365971801, loss: 0.48654089537617096
epoch : 300, w1: 4.4721795220959155, w2: 1.438787986553552, bias: -2.7073878119223735, loss: 0.4750814640632793
epoch : 350, w1: 4.917245868007634, w2: 1.4525660781176122, bias: -2.901176333556766, loss: 0.46561475306999006
epoch : 366, w1: 5.051047623653049, w2: 1.4569794548473887, bias: -2.9596534546250037, loss: 0.462939440958

(5.051047623653049, 1.4569794548473887, -2.9596534546250037)

In [14]:
custom_model.predict (X_test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64